# ABECTO Tutorial

ABECTO is an **AB**ox **E**valuation and **C**omparison **T**ool for **O**ntologies. It allows to easily compare and evaluate two or more RDF knowledge bases regarding the contained information. This tutorial provides an introduction to the use of ABECTO.


## Preparation

Before we can start, we need to do a few preparation steps.

ABECTO is running as a HTTP REST service in the background. We will use some provided functions, which hide the raw HTTP requests.

In [1]:
from abecto import *

First, we create some sample files that we will use in this tutorial.

In [2]:
import tempfile

source1file1 = tempfile.TemporaryFile(mode = "w+")
source1file1.write("""
    BASE         <http://example.org/a/>
    PREFIX :     <http://example.org/a/>
    PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>

    :alice rdf:type   :Person ;
           rdfs:label "Alice" ;
           :pnr       "12345"^^xsd:integer .
""")
source1file1.seek(0)

source1file2 = tempfile.TemporaryFile(mode = "w+")
source1file2.write("""
    BASE         <http://example.org/a/>
    PREFIX :     <http://example.org/a/>
    PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>

    :bob rdf:type   :Person ;
         rdfs:label "Bob" ;
         :pnr       "67890"^^xsd:integer .
""")
source1file2.seek(0)

source2file1 = tempfile.TemporaryFile(mode = "w+")
source2file1.write("""
    BASE         <http://example.org/b/>
    PREFIX :     <http://example.org/b/>
    PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    :alice rdf:type   :Person ;
           rdfs:label "Alice" .

    :bob rdf:type   :Person ;
         rdfs:label "Bob" .

    :charlie rdf:type   :Person ;
             rdfs:label "Charlie" .
""")
source2file1.seek(0);

source3file1 = tempfile.TemporaryFile(mode = "w+")
source3file1.write("""
    BASE         <http://example.net/b/>
    PREFIX :     <http://example.net/b/>
    PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    :P001 rdf:type   :Person ;
           rdfs:label "Alice" .

    :P002 rdf:type   :Person ;
             rdfs:label "Charlie" .
""")
source3file1.seek(0);

Now, we start the service. **This might take a few seconds.**

In [3]:
abecto = Abecto("http://localhost:8080/", "target/abecto.jar")
abecto.start()

After the service was started, we are ready to create our ontology evaluation and comparison project.

## Project Setup

First, we create a new ABECTO project. We can also give the project an arbitrary name.

In [4]:
project = abecto.project("My Project")

Now, we see our new project in the list of projects.

In [5]:
#abecto.projects()

We can also request information of a certain project.

In [6]:
#getProject(project)

Furthermore, we can delete projects.

In [7]:
#trashProject <- createProject("Trash Project")
#deleteProject(trashProject)
#listProjects()

Now, we create a knowledge base object for each knwoledge base we want to include into the project.

In [8]:
kb1 = project.knowledgeBase("Knowledge Base ABCDEFG")
kb2 = project.knowledgeBase("Knowledge Base HIJKLMN")
kb3 = project.knowledgeBase("Knowledge Base OPQRSTU")

The knowledge bases appear now in the list of knowledge bases of the project.

In [9]:
#project.knowledgeBases()

We can also request information of a certain knowledge base.

In [10]:
#getKnowledgebase(kb1)

And we can delete knowledge bases.

In [11]:
#trashKB <- createKnowledgebase(project,"Trash Knowledge Base")
#deleteKnowledgebase(trashKB)
#listKnowledgebases(project)

In [12]:
kb1source1 = kb1.source("RdfFileSourceProcessor")
kb1source2 = kb1.source("RdfFileSourceProcessor")
kb2source1 = kb2.source("RdfFileSourceProcessor")
kb3source1 = kb3.source("RdfFileSourceProcessor")

In [13]:
#getStep(kb1source1)

In [14]:
#listSteps(project)

In [15]:
kb1source1.load(source1file1)
kb1source2.load(source1file2)
kb2source1.load(source2file1)
kb3source1.load(source3file1);

In [16]:
personPattern = {"person": "?person <http://www.w3.org/2000/01/rdf-schema#label> ?label ."}
categories1 = kb1source1.plus(kb1source2).into("ManualCategoryProcessor", {"patterns": personPattern})
categories2 = kb2source1.into("ManualCategoryProcessor", {"patterns": personPattern})
categories3 = kb3source1.into("ManualCategoryProcessor", {"patterns": personPattern})

In [17]:
mapping = categories1.plus(categories2).plus(categories3).into("JaroWinklerMappingProcessor", {"threshold": 0.9, "case_sensitive": False, "category": "person", "variables": ["label"]})

In [18]:
mappingReport = mapping.into("MappingReportProcessor")

In [19]:
project.runAndAwait()

In [20]:
mappingReport.lastProcessing().graphAsDataFrame()

,@id,@type,category1,category2,data1,data2,id1,id2,knowledgeBase1,knowledgeBase2
0,_:b0,http://fusion.cs.uni-jena.de/ontology/abecto/M...,person,person,"{""label"":""Charlie"",""person"":""http://example.or...",NaN,http://example.org/b/charlie,NaN,b32d8ad8-e14e-432e-8627-d8b319ec1edc,4267b806-7327-419a-ad31-be39b766e1c4
1,_:b1,http://fusion.cs.uni-jena.de/ontology/abecto/M...,person,person,"{""label"":""Alice"",""person"":""http://example.org/...","{""label"":""Alice"",""person"":""http://example.net/...",http://example.org/b/alice,http://example.net/b/P001,b32d8ad8-e14e-432e-8627-d8b319ec1edc,43142f30-1a45-43f1-97ba-3db8de9bcdb3
2,_:b2,http://fusion.cs.uni-jena.de/ontology/abecto/M...,person,person,NaN,"{""label"":""Bob"",""person"":""http://example.org/a/...",NaN,http://example.org/a/bob,43142f30-1a45-43f1-97ba-3db8de9bcdb3,4267b806-7327-419a-ad31-be39b766e1c4
3,_:b3,http://fusion.cs.uni-jena.de/ontology/abecto/M...,person,person,"{""label"":""Alice"",""person"":""http://example.org/...","{""label"":""Alice"",""person"":""http://example.org/...",http://example.org/b/alice,http://example.org/a/alice,b32d8ad8-e14e-432e-8627-d8b319ec1edc,4267b806-7327-419a-ad31-be39b766e1c4
4,_:b4,http://fusion.cs.uni-jena.de/ontology/abecto/M...,person,person,"{""label"":""Alice"",""person"":""http://example.net/...","{""label"":""Alice"",""person"":""http://example.org/...",http://example.net/b/P001,http://example.org/a/alice,43142f30-1a45-43f1-97ba-3db8de9bcdb3,4267b806-7327-419a-ad31-be39b766e1c4
5,_:b5,http://fusion.cs.uni-jena.de/ontology/abecto/M...,person,person,"{""label"":""Bob"",""person"":""http://example.org/b/...",NaN,http://example.org/b/bob,NaN,b32d8ad8-e14e-432e-8627-d8b319ec1edc,43142f30-1a45-43f1-97ba-3db8de9bcdb3
6,_:b6,http://fusion.cs.uni-jena.de/ontology/abecto/M...,person,person,"{""label"":""Charlie"",""person"":""http://example.ne...",NaN,http://example.net/b/P002,NaN,43142f30-1a45-43f1-97ba-3db8de9bcdb3,4267b806-7327-419a-ad31-be39b766e1c4
7,_:b7,http://fusion.cs.uni-jena.de/ontology/abecto/M...,person,person,"{""label"":""Bob"",""person"":""http://example.org/b/...","{""label"":""Bob"",""person"":""http://example.org/a/...",http://example.org/b/bob,http://example.org/a/bob,b32d8ad8-e14e-432e-8627-d8b319ec1edc,4267b806-7327-419a-ad31-be39b766e1c4
8,_:b8,http://fusion.cs.uni-jena.de/ontology/abecto/M...,person,person,"{""label"":""Charlie"",""person"":""http://example.or...","{""label"":""Charlie"",""person"":""http://example.ne...",http://example.org/b/charlie,http://example.net/b/P002,b32d8ad8-e14e-432e-8627-d8b319ec1edc,43142f30-1a45-43f1-97ba-3db8de9bcdb3


In [21]:
mappingReport.info()

{'id': '81e8c2ec-850e-42ed-ae82-4756074cc49e',
 'project': 'ae3a3939-f0db-4d3e-9c03-c7e1260f4d99',
 'knowledgeBase': None,
 'parameter': {'id': '7afd3687-2b09-432d-b022-21a501748af3', 'parameters': {}},
 'processorClass': 'de.uni_jena.cs.fusion.abecto.processor.implementation.MappingReportProcessor',
 'mapping': False,
 'meta': True,
 'refinement': True,
 'source': False,
 'transformation': False}

In [22]:
mappingReport.lastProcessing().report()